\# Developer: Ali Hashaam (ali.hashaam@initos.com) <br>
\# 18th December 2018 <br>

\# © 2019 initOS GmbH <br>
\# License MIT <br>

To detect multiword phrases in sentences within a dataset,
we are going to use gensim's model.phrases class https://radimrehurek.com/gensim/models/phrases.html
to do that first we make have to make an object of the model.phrases class and utilize the method add\_vocab
to add all the tokenized sentences in it! and we will store the object with in a pickle file so that when we
want to detect multiwords in sentences within the dataset under consideration, we will read pickle file 
and pass our sentences through the model.phrases object (we just read from pickle file) and it will put 
\_ be the multiwords, making new york as new\_york (considering if new_york is present frequently inside the dataset)

In [6]:
import pandas as pd
import numpy as np
import re, time, string
import nltk
import pickle
from gensim.models import Phrases
from gensim.models import Word2Vec
from nltk.corpus import stopwords

In [4]:
def add_sentence_in_vocab(row, textual_columns):
    """tokenize and remove stopwords and then add the sentence in model.phrase object"""
    global bigram
    for col in textual_columns:
        sentence = row[col].translate(None, string.punctuation)
        sentence = unicode(sentence, "utf-8")
        tokenize = nltk.word_tokenize(sentence)
        stopWords = set(nltk.corpus.stopwords.words('english'))
        sentence = [x for x in tokenize if x not in stopWords]
        if len(sentence):
            bigram.add_vocab([sentence])
    return row

In [5]:
def process_data(df, textual_columns):
    df = df.apply(lambda x: add_sentence_in_vocab(x, textual_columns), axis=1)
    return df

In [8]:
if __name__ == '__main__':
    file_directory = "../datasets/"
    csv_files = ['mantis_bug_notes', 'mantis_bugs']
    #csv_files = ['mantis_en_issues'] 
    gensim_directory = "../datasets/gensim_multiphrases/"
    bigram = Phrases()
    for csv_file in csv_files:
        df = pd.read_csv(file_directory + csv_file +'.csv').head(5)
        if 'mantis_bug_notes' in csv_file:
            textual_cols = ['bug_note']
            id_col = 'bugnote_id'
        elif 'mantis_bugs' in csv_file:
            textual_cols = ['summary', 'description', 'additional_information', 'steps_to_reproduce']
            id_col = 'id'
        df = process_data(df, textual_cols)
    with open("{}{}.pkl".format(gensim_directory, '_'.join(csv_files)), "wb") as fp:   #Pickling
        pickle.dump(bigram, fp)

In [ ]:
#filename|textual_columns (comma seperated)|id_column